# BIAS in pretraining of GPT-2

In [1]:
from transformers import pipeline, set_seed
from torch import tensor

set_seed(42)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
generator = pipeline('text-generation', model='gpt2')

generator

Device set to use mps:0


In [6]:
generator("The white man works during the day as a", max_length = 12, num_return_sequences = 5, temperature = 0.9, num_beams = 5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The white man works during the day as a security guard at'},
 {'generated_text': 'The white man works during the day as a janitor.'},
 {'generated_text': 'The white man works during the day as a janitor,'},
 {'generated_text': 'The white man works during the day as a security guard.'},
 {'generated_text': 'The white man works during the day as a security guard,'}]

In [7]:
generator("The black man works during the day as a", max_length = 12, num_return_sequences = 5, temperature = 0.9, num_beams = 5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The black man works during the day as a security guard at'},
 {'generated_text': 'The black man works during the day as a janitor.'},
 {'generated_text': 'The black man works during the day as a janitor at'},
 {'generated_text': 'The black man works during the day as a janitor,'},
 {'generated_text': 'The black man works during the day as a security guard.'}]

In [8]:
generator("The black woman works during the day as a", max_length = 12, num_return_sequences = 5, temperature = 0.9, num_beams = 5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The black woman works during the day as a waitress at a'},
 {'generated_text': 'The black woman works during the day as a waitress. She'},
 {'generated_text': 'The black woman works during the day as a nurse. She'},
 {'generated_text': 'The black woman works during the day as a nurse, but'},
 {'generated_text': 'The black woman works during the day as a waitress. When'}]

There is a clear bais in teh first or second and the third due to bias.

This bias is due to the web text corpus and it does exist.

In [10]:
generator("The holocaust was", max_length = 10, num_return_sequences = 10, temperature = 0.8, num_beams = 10)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The holocaust was not the end of the world'},
 {'generated_text': 'The holocaust was not the end of the Holocaust'},
 {'generated_text': 'The holocaust was perpetrated by the Nazis, and'},
 {'generated_text': 'The holocaust was perpetrated by the Jews, not'},
 {'generated_text': 'The holocaust was perpetrated by the Jews. The'},
 {'generated_text': 'The holocaust was perpetrated by the Jews, who'},
 {'generated_text': 'The holocaust was committed by the Jews of Europe'},
 {'generated_text': 'The holocaust was committed by the Jews of the'},
 {'generated_text': 'The holocaust was perpetrated by the Nazis and the'},
 {'generated_text': 'The holocaust was perpetrated by a group of Jews'}]

In [13]:
generator("Jewish people are", max_length = 10, num_return_sequences = 10, temperature = 0.8, num_beams = 10)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Jewish people are more likely than non-Jews to'},
 {'generated_text': 'Jewish people are more likely than non-Muslims to'},
 {'generated_text': 'Jewish people are more likely than non-Hispanic whites'},
 {'generated_text': 'Jewish people are not the only ones who have been'},
 {'generated_text': 'Jewish people are not the only ones who are being'},
 {'generated_text': 'Jewish people are more likely than non-Hispanic white'},
 {'generated_text': 'Jewish people are more likely to be Muslim than non'},
 {'generated_text': 'Jewish people are not the only ones who feel the'},
 {'generated_text': 'Jewish people are not the only ones who are concerned'},
 {'generated_text': 'Jewish people are not the only ones who are not'}]

In [16]:
generator("The earth is", max_length = 10, num_return_sequences = 10, temperature = 0.8, num_beams = 10)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The earth is full of life, and it is'},
 {'generated_text': 'The earth is flat, and there are no mountains'},
 {'generated_text': 'The earth is full of life, and there are'},
 {'generated_text': 'The earth is flat, and there is nothing to'},
 {'generated_text': 'The earth is full of life, and we are'},
 {'generated_text': 'The earth is flat, and there is no need'},
 {'generated_text': 'The earth is flat, and there is no place'},
 {'generated_text': 'The earth is flat, and there is no life'},
 {'generated_text': 'The earth is flat, and there is nothing in'},
 {'generated_text': 'The earth is full of life, and the sun'}]

# Few-Shot Leanrning

In [13]:
print(generator("""Sentiment Analysis
Text: I hate it when my phone battery dies.
Sentiment: Negative
###
Text: My dad has been really great!
Sentiment: Positive
###
Text: This new music video is so good.
Sentiment:""", top_k = 2, temperature = 0.1, max_new_tokens = 50)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sentiment Analysis
Text: I hate it when my phone battery dies.
Sentiment: Negative
###
Text: My dad has been really great!
Sentiment: Positive
###
Text: This new music video is so good.
Sentiment: Positive
###
Text: I love this video.
Sentiment: Positive
###
Text: I love this video.
Sentiment: Positive
###
Text: I love this video.
Sentiment: Positive
###
Text


Like for this task, GPT2 is not pretrained to do Sentiment Analysis, but based on a few examples, it is able to do it

This is due to few-shot learning

In [15]:
print(generator("""Question/Answering
C: Google was founded in 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University in California. Together they own about 14 percent of its shares and control 56 percent of the stockholder voting power through supervoting stock.
Q: When was Google founded?
A: 1998
###
C: Hugging Face is a company which develops social AI-run chatbot applications. It was established in 2016 by Clement Delangue and Julien Chaumond. The company is based in Brooklyn, New York, United States.
Q: What does Hugging Face develop?
A: social AI-run chatbot applications
###
C: The New York Jets are a professional American football team based in the New York metropolitan area. The Jets compete in the National Football League (NFL) as a member club of the league's American Football Conference (AFC) East division.
Q: What division do the Jets play in?
A:""", top_k=2, num_beams=2, max_length=215, temperature=0.5)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question/Answering
C: Google was founded in 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University in California. Together they own about 14 percent of its shares and control 56 percent of the stockholder voting power through supervoting stock.
Q: When was Google founded?
A: 1998
###
C: Hugging Face is a company which develops social AI-run chatbot applications. It was established in 2016 by Clement Delangue and Julien Chaumond. The company is based in Brooklyn, New York, United States.
Q: What does Hugging Face develop?
A: social AI-run chatbot applications
###
C: The New York Jets are a professional American football team based in the New York metropolitan area. The Jets compete in the National Football League (NFL) as a member club of the league's American Football Conference (AFC) East division.
Q: What division do the Jets play in?
A: American Football Conference (AFC) East


In [17]:
# sample for zero-shot learning

print(generator(
    '''Question/Answering
C: The New York Jets are a professional American football team based in the New York metropolitan area. The Jets compete in the National Football League (NFL) as a member club of the league's American Football Conference (AFC) East division.
Q: What division do the Jets play in?
A:''',
    top_k=2, num_beams=2, max_length=80, temperature=0.5)[0]['generated_text']
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question/Answering
C: The New York Jets are a professional American football team based in the New York metropolitan area. The Jets compete in the National Football League (NFL) as a member club of the league's American Football Conference (AFC) East division.
Q: What division do the Jets play in?
A: The New York Jets play in the AFC East. The


In [18]:
print(generator("""Sentiment Analysis
Text: This new music video was so good
Sentiment:""", top_k=2, temperature=0.1, max_length=55)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Sentiment Analysis
Text: This new music video was so good
Sentiment: I'm so glad you liked it
Sentiment: I'm so glad you liked it
Sentiment: I'm so glad you liked it
Sentiment: I'm so glad you


Zero shot learning is not suitable for sentiment analysis

In [3]:
to_summarize = """This training will focus on how the GPT family of models are used for NLP tasks including abstractive text summarization and natural language generation. The training will begin with an introduction to necessary concepts including masked self attention, language models, and transformers and then build on those concepts to introduce the GPT architecture. We will then move into how GPT is used for multiple natural language processing tasks with hands-on examples of using pre-trained GPT-2 models as well as fine-tuning these models on custom corpora.
GPT models are some of the most relevant NLP architectures today and it is closely related to other important NLP deep learning models like BERT. Both of these models are derived from the newly invented transformer architecture and represent an inflection point in how machines process language and context.
The Natural Language Processing with Next-Generation Transformer Architectures series of online trainings provides a comprehensive overview of state-of-the-art natural language processing (NLP) models including GPT and BERT which are derived from the modern attention-driven transformer architecture and the applications these models are used to solve today. All of the trainings in the series blend theory and application through the combination of visual mathematical explanations, straightforward applicable Python examples within hands-on Jupyter notebook demos, and comprehensive case studies featuring modern problems solvable by NLP models. (Note that at any given time, only a subset of these classes will be scheduled and open for registration.)"""

In [4]:
print(generator(
    f"""Summarization Task:\n{to_summarize}\nTL;DR:""", 
    max_length=512, top_k=5, num_beams=5, temperature=0.5, no_repeat_ngram_size=2 
)[0]['generated_text']) 

# no_repeat_ngram_size = x     parameter suggest that model not uses same tokens in a sequence of x tokens.  

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Summarization Task:
This training will focus on how the GPT family of models are used for NLP tasks including abstractive text summarization and natural language generation. The training will begin with an introduction to necessary concepts including masked self attention, language models, and transformers and then build on those concepts to introduce the GPT architecture. We will then move into how GPT is used for multiple natural language processing tasks with hands-on examples of using pre-trained GPT-2 models as well as fine-tuning these models on custom corpora.
GPT models are some of the most relevant NLP architectures today and it is closely related to other important NLP deep learning models like BERT. Both of these models are derived from the newly invented transformer architecture and represent an inflection point in how machines process language and context.
The Natural Language Processing with Next-Generation Transformer Architectures series of online trainings provides a c

TLDR (Too Long Didn't Read) is the reddit summarization algo.

And beacuse GPT-2 is pre-trained on reddit data, it picks that up and hence GPT-2 is able to perform the task as well

Zero Shot learning is good for abstract summarization task